<a href="https://colab.research.google.com/github/chetuchetan1/7th-Sem-ML-LAB/blob/master/candidate_elimination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import csv
a = [] #empty list
with open("/content/findsss.csv") as trainData:
  for row in csv.reader(trainData):
    a.append (row)
    print(row)
n = len(a[0])-1

['Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same', 'Yes']
['Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same', 'Yes']
['Cloudy', 'Cold', 'High', 'Strong', 'Warm', 'Same', 'No']
['Sunny', 'Cold', 'High', 'Strong', 'Cold', 'Change', 'Yes']
